# L.A. hospital occupancy data over time

#### An analysis of HHS hospital data, by @datagraphics and @stiles.

#### **Questions?** [matt.stiles@latimes.com](matt.stiles@latimes.com) \\ 310.529.8749

---

### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt

%matplotlib inline
import json
import numpy as np
import altair as alt
import altair_latimes as lat

alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

### Fetch timeseries data

In [3]:
df = pd.read_csv(
    "output/trimmed_timeseries.csv",
    parse_dates=["week"],
    infer_datetime_format=True,
    dtype={"fips": str},
)

In [4]:
df.head()

,hospital,hospital_subtype,ccn,week,county,fips,total_beds,total_staffed_adult_icu_beds,total_covid_icu_patients,total_available_adult_icu_beds,pct_occupied_adult_icu_beds,total_covid_patients,all_patients,covid_patients_share,region
0,Kaiser Foundation Hospital - Fremont,Short Term,050512,2021-01-29,Alameda,001,94.0,12.0,8.0,1.0,0.92,34.0,85.0,0.40,Bay Area
1,Kaiser Foundation Hospital - Oakland/Richmond,Short Term,050075,2021-01-29,Alameda,001,270.0,31.0,22.0,1.0,0.97,64.0,248.0,0.26,Bay Area
2,Washington Hospital,Short Term,050195,2021-01-29,Alameda,001,230.0,29.0,15.0,0.0,1.00,59.0,196.0,0.30,Bay Area
3,Kaiser Foundation Hospital - San Leandro,Short Term,050777,2021-01-29,Alameda,001,182.0,19.0,11.0,1.0,0.95,42.0,165.0,0.25,Bay Area
4,St Rose Hospital,Short Term,050002,2021-01-29,Alameda,001,95.0,14.0,6.0,2.0,0.86,24.0,58.0,0.41,Bay Area


In [5]:
len(df)

9360

---

### How have L.A. hositals changed over time? 

In [16]:
surge_la = df[(df["week"] > "2020-11-13") & (df["county"] == "Los Angeles")]

In [22]:
surge_la[surge_la["week"] == surge_la["week"].max()][
    ["hospital", "week", "total_beds", "total_covid_patients", "covid_patients_share"]
].sort_values("covid_patients_share", ascending=False)

,hospital,week,total_beds,total_covid_patients,covid_patients_share
133,Coast Plaza Hospital,2021-01-29,72.0,33.0,0.72
86,Pih Hospital - Downey,2021-01-29,162.0,83.0,0.70
119,East Los Angeles Doctors Hospital,2021-01-29,102.0,23.0,0.64
109,Kaiser Foundation Hospital - Baldwin Park,2021-01-29,248.0,113.0,0.60
118,Kaiser Foundation Hospital - Downey,2021-01-29,385.0,142.0,0.59
70,Pacifica Hospital of The Valley,2021-01-29,142.0,56.0,0.57
89,San Dimas Community Hospital,2021-01-29,162.0,31.0,0.54
74,Greater El Monte Community Hospital,2021-01-29,86.0,26.0,0.54
85,Palmdale Regional Medical Center,2021-01-29,184.0,70.0,0.53
76,"Martin Luther King, Jr. Community Hospital",2021-01-29,260.0,112.0,0.53


In [14]:
alt.Chart(surge_la).mark_line().encode(
    x=alt.X("week:T", axis=alt.Axis(format="", title=" ", tickCount=3)),
    y=alt.Y(
        "covid_patients_share",
        title=" ",
        axis=alt.Axis(format="%", title="", tickCount=6),
    ),
    facet=alt.Facet("hospital", columns=10),
).properties(
    width=100, height=100, title="Share of all patients with Covid in SoCal Counties"
)

alt.Chart(...)